<a href="https://colab.research.google.com/github/deepakawl/supplychain-analytics-teaching/blob/main/Optimal_Facility_Location_OpenMapsAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimal Facility Location

---
P&G is opening a new location in the Eastern US. The proposed plant will have three major suppliers and will be serving five major markets. Estimated monthly inbound and outbound freight volumes and transportation costs are given.

<!--
|         |Location        |$/Ton Mile| Tons |
|         |Markets         |    F_n   |  W_n |
|      ---|             ---|       ---|   ---|
|         |Buffalo, NY     |     0.18 |  500 |
| Sources |Memphis, TN     |     0.19 |  300 |
|         |St.Louis, MO    |     0.17 |  700 |
|         |Atlanta, GA     |     0.30 |  250 |
|         |Boston, MA      |     0.30 |  150 |
| Markets |Jacksonville, FL|     0.30 |  250 |
|         |Philadelphia, PA|     0.30 |  175 |
|         |New York, NY    |     0.30 |  300 | -->

Propose a feasible location that minimizes the monthly cost of inbound and outbound transportation and estimate that cost.

In [1]:
import pandas as pd

In [2]:
# Creating the DataFrame
data = pd.DataFrame({
    'Location': ['Buffalo, NY', 'Memphis, TN', 'St.Louis, MO', 'Atlanta, GA', 'Boston, MA', 'Jacksonville, FL', 'Pittsburgh, PA', 'New York, NY'],
    'F': [0.18, 0.19, 0.17, 0.30, 0.30, 0.30, 0.30, 0.30 ],
    'W': [500, 300, 700, 250, 150, 250, 175, 300]
    })

data

,Location,F,W
0,"Buffalo, NY",0.18,500
1,"Memphis, TN",0.19,300
2,"St.Louis, MO",0.17,700
3,"Atlanta, GA",0.30,250
4,"Boston, MA",0.30,150
5,"Jacksonville, FL",0.30,250
6,"Pittsburgh, PA",0.30,175
7,"New York, NY",0.30,300


## Install and import required packages

In [3]:
# !pip install googlemaps
!pip install polyline
!pip install folium
!pip install geopy requests


import math
import scipy.optimize as opt
import polyline
import folium

In [4]:
import time
import requests
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="location_optimization_app")

## Function to geocode a location

In [5]:
# Function to geocode an address using OpenStreetMap's Nominatim service.
def get_geocode(location, delay=1):
    """
    Geocode an address using geopy's Nominatim.

    Parameters:
    - address: String containing the address to geocode.
    - geolocator: A Nominatim instance.
    - delay: Seconds to wait between requests (to respect usage policy).

    Returns:
    - Tuple (latitude, longitude) or (None, None) if the address cannot be found.
    """
    try:
        coords = geolocator.geocode(location, timeout=10)
        time.sleep(delay)  # Respect the usage limits of Nominatim.
        if coords is None:
            print(f"Warning: No geocode result for {location}.")
            return None, None
        lat = round(coords.latitude, 4)
        lng = round(coords.longitude, 4)
        return lat, lng
    except Exception as e:
        print(f"Error geocoding {location}: {e}")
        return None, None

In [9]:
# Get geocodes for each address in the DataFrame
df = data.copy()
df[['Lat', 'Lng']] = data['Location'].apply(lambda x: pd.Series(get_geocode(x)))

df

,Location,F,W,Lat,Lng
0,"Buffalo, NY",0.18,500,42.8867,-78.8784
1,"Memphis, TN",0.19,300,35.1460,-90.0518
2,"St.Louis, MO",0.17,700,38.6280,-90.1910
3,"Atlanta, GA",0.30,250,33.7545,-84.3898
4,"Boston, MA",0.30,150,42.3554,-71.0605
5,"Jacksonville, FL",0.30,250,30.3322,-81.6557
6,"Pittsburgh, PA",0.30,175,40.4417,-79.9901
7,"New York, NY",0.30,300,40.7127,-74.0060


## Approach 1: Optimize distance and cost using Haversine (Great Circle) distance

In [10]:
## Function to calculate distance between two geocodes and compute total cost

# Great circle ("as crow flies") distance
def calc_dist_haversine(lat1, lng1, lat2, lng2):
    """
    Calculate haversine distance between two geocodes.

    Parameters:
    - lat1, lng1: Latitude and longitude of the first location.
    - lat2, lng2: Latitude and longitude of the second location

    Returns:
    - Haversine distance in miles.
    """
    # Convert latitude and longitude from degrees to radians
    lat1, lng1, lat2, lng2 = map(math.radians, [lat1, lng1, lat2, lng2])

    a = math.sin((lat2 - lat1) / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin((lng2 - lng1) / 2) ** 2
    dist_haversine_miles = 3959 * 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return dist_haversine_miles

# Function to calculate Haversine distance from a point to locations in dataframe and total cost given distances
def calc_cost_haversine(coords, data):
    lat, lng = coords
    distances = []

    for _, row in data.iterrows():
        distances.append(calc_dist_haversine(lat, lng, row['Lat'], row['Lng']))
    cost = (data['F'] * data['W'] * pd.Series(distances)).sum()

    return cost

In [11]:
# Initial guess for the optimization algorithm (somewhere close to the average of latitudes and longitudes)
initial_guess = [df['Lat'].mean(), df['Lng'].mean()]

# Extract min and max values of Latitude and Longitude columns for bounds
# bounds = [(df['Lat'].min() - 5, df['Lat'].max() + 5), (df['Lng'].min() - 5, df['Lng'].max() + 5)]

# Minimize the total distance function using scipy's minimize
# result = opt.minimize(calc_cost_haversine, initial_guess, args=(df,), method='SLSQP', bounds=bounds)
result_haversine_geocode = opt.minimize(calc_cost_haversine, initial_guess, args=(df,), method='SLSQP')

# Get the location address
result_haversine_address = geolocator.reverse((result_haversine_geocode.x[0], result_haversine_geocode.x[1])).address

print(f"\nMinimum cost and corresponding optimal location (lat, long): \n$ {(result_haversine_geocode.fun):,.2f}; { (result_haversine_geocode.x).round(4) }\nAddress: {result_haversine_address}")

# Minimum cost and corresponding optimal location (lat, long):
# $ 259,123.43; [ 38.9543 -81.6461]
# Address: 1399, Drift Run Road, Drift Run, Jackson County, West Virginia, 25275, United States


Minimum cost and corresponding optimal location (lat, long): 
$ 259,096.23; [ 38.9542 -81.6467]
Address: Drift Run Road, Drift Run, Jackson County, West Virginia, United States


In [12]:
# Create a map centered around the optimal location
mhvs = folium.Map(location = result_haversine_geocode.x, zoom_start = 5)

# Add a marker for the optimal location
folium.Marker(location = result_haversine_geocode.x,
              popup = folium.Popup(f"Optimal Location <br> Cost: ${result_haversine_geocode.fun:,.2f} <br> Address: {result_haversine_address}", max_width=500),
              icon = folium.Icon(color='orange', icon='star')).add_to(mhvs)

# Add markers for each location in the DataFrame
# Add lines connecting the optimal location to each point
for index, row in df.iterrows():
    folium.Marker(location = [row['Lat'], row['Lng']],
                  popup = folium.Popup(row['Location'], max_width=500),
                  icon = folium.Icon(color = 'blue')).add_to(mhvs)

    folium.PolyLine(locations = [[row['Lat'], row['Lng']], result_haversine_geocode.x],
                    color = 'blue',
                    weight = 2.5,
                    opacity = 1).add_to(mhvs)

# Save the map as an HTML file
#mhvs.save("map_with_locations_and_paths.html")

In [13]:
display(mhvs)

## Approach 2: Optimize distance and cost using Travel distance

In [14]:
# Function to calculate travel distance (in miles) using OSRM API
def calc_dist_driving(lat1, lng1, lat2, lng2, delay=0):
    """
    Calculate travel distance between two geocodes using OSRM.

    Parameters:
    - lat1, lng1: Latitude and longitude of the first location.
    - lat2, lng2: Latitude and longitude of the second location
    - delay: Seconds to wait between requests (to respect usage policy).

    Returns:
    - Driving distance in miles.
    """
    # OSRM expects coordinates as: longitude,latitude.
    url = f"http://router.project-osrm.org/route/v1/driving/{lng1},{lat1};{lng2},{lat2}?overview=false"

    try:
        response = requests.get(url, timeout=5)
        data = response.json()
        if "routes" in data and len(data["routes"]) > 0:
            # OSRM returns distance in meters
            dist_meters = data["routes"][0]["distance"]
            dist_miles = dist_meters / 1609.344
            return dist_miles
        else:
            print("Warning: No route found between the given locations via OSRM.")
            return calc_dist_haversine(lat1, lng1, lat2, lng2)  # Fallback to Haversine

    except Exception as e:
        print(f"Error retrieving OSRM data: {e}. Using Haversine distance as fallback.")

        return calc_dist_haversine(lat1, lng1, lat2, lng2)

# Function to calculate cost using driving distances from a point to locations in a DataFrame and total cost given distances
def calc_cost_driving(coords, data):
    lat, lon = coords
    distances = []

    for _, row in data.iterrows():
        distances.append(calc_dist_driving(lat, lon, row['Lat'], row['Lng']))
    cost = (data['F'] * data['W'] * pd.Series(distances)).sum()

    return cost

In [15]:
### Calculate driving distance using OpenMaps Direction API

# Define ranges (boundary and increment) based on optimal point estimated using Haversine distance method
increment = 1
diff = 1
ranges = (slice(int(result_haversine_geocode.x[0]) - diff, round(result_haversine_geocode.x[0], 0) + diff, increment),
          slice(int(result_haversine_geocode.x[1]) - diff, round(result_haversine_geocode.x[1], 0) + diff, increment))

# Minimize the total distance function using scipy's integer minimizer
result_driving_geocode = opt.brute(calc_cost_driving, ranges, args=(df,), full_output = True, finish = None)

# Reverse geocode the optimal point found by driving distance optimization
result_driving_address = geolocator.reverse((result_driving_geocode[0][0], result_driving_geocode[0][1])).address

print(f"\nMinimum cost, Travel time, and corresponding optimal location (lat, long): \n$ {result_driving_geocode[1]:,.2f}; {result_driving_geocode[0].tolist() }\nAddress: {result_driving_address}")

# Minimum cost, Travel time, and corresponding optimal location (lat, long):
# $ 317,949.69; [39.0, -82.0]
# Address: High Street, Hartford City, Mason County, West Virginia, 25247, United States


Minimum cost, Travel time, and corresponding optimal location (lat, long): 
$ 317,944.22; [39.0, -82.0]
Address: High Street, Hartford City, Mason County, West Virginia, 25247, United States


## Visualize the results on map

In [18]:
### Code to print locations on map

# Create a map centered around the optimal location
mdrv = folium.Map(location = result_driving_geocode[0], zoom_start = 5)

# Add a marker for the optimal location (both Haversine and Travel distance)
## Haversine distance
folium.Marker(location = result_haversine_geocode.x,
              popup = folium.Popup(f"Optimal Location (Haversine)<br>Cost: ${result_haversine_geocode.fun:,.2f}<br>Address: {result_haversine_address}", max_width=500),
              icon = folium.Icon(color='orange', icon='star')).add_to(mdrv)

## Driving distance
folium.Marker(location = result_driving_geocode[0],
              popup = folium.Popup(f"Optimal Location (Driving)<br>Cost: ${result_driving_geocode[1]:,.2f}<br>Address: {result_driving_address}", max_width=500),
              icon = folium.Icon(color='green', icon='star')).add_to(mdrv)

# Draw driving paths between locations
optimal_point = tuple(result_driving_geocode[0])

for index, row in df.iterrows():
    # Add markers for locations from DataFrame
    folium.Marker(location = (row['Lat'], row['Lng']),
                  popup = folium.Popup(row['Location'], max_width=500),
                  icon = folium.Icon(color = 'blue')).add_to(mdrv)

    # Get directions using OSRM API (OpenMaps alternative)
    url = f"http://router.project-osrm.org/route/v1/driving/{optimal_point[1]},{optimal_point[0]};{row['Lng']},{row['Lat']}?overview=full&geometries=polyline"
    try:
        response = requests.get(url, timeout=10)
        data = response.json()
        if "routes" in data and len(data["routes"]) > 0:
            encoded_poly = data["routes"][0]["geometry"]
            # Extract polyline points from API response and add to Folium map
            points = polyline.decode(encoded_poly)
            # Add driving path on the map
            folium.PolyLine(locations = points, color = 'blue', weight = 2.5, opacity = 1).add_to(mdrv)
        else:
            print(f"Warning: No route found from {optimal_point} to {(row['Lat'], row['Lng'])} via OSRM.")
    except Exception as e:
        print(f"Error retrieving route from {optimal_point} to {(row['Lat'], row['Lng'])}: {e}")

# Save the map as an HTML file
#mdrv.save("map_with_locations_and_paths.html")

In [19]:
display(mdrv)